In [ ]:
import sys
import numpy as np
from numpy import loadtxt
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import train_test_split
import pandas as pd 
debug = True

f1_scores = []
fpr = []
tpr = []

print('Synthetic Dataset PCA')

synthetic = pd.read_csv("/kaggle/input/synthetic-dataset-with-pca/Synthetic_Dataset_with_PCA.csv", header = None, delimiter = ',')
outputs = pd.read_csv("/kaggle/input/combined-dataset-before-smote/output-2.csv", header = None, delimiter = ',') 
print(synthetic)

X = synthetic.iloc[1:]
y = outputs.filter([26,27,28,29])
print(X,y)

unique_rows = np.unique(y, axis=0).tolist()
print(unique_rows)

y['combined']= y.values.tolist()
y = y.drop(columns = [26,27,28,29])
print(y)
print(y.shape)
#merges mutliple targets into single multiclassfication for y
i = 0
while i < len(y):
    j = 0
    while j < len(unique_rows):
        if y.loc[y.index[i],'combined'] == unique_rows[j]:
            y.loc[y.index[i],'combined'] = unique_rows.index(unique_rows[j])
        j +=1
    i+=1
    
y['combined'] = y['combined'].astype(int)

y.to_numpy()

print(y)


# Split dataset into training and validation sets
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("done")
print(x_train.shape)
print(y_train.shape)



In [ ]:
from sklearn.svm import SVC

from sklearn.preprocessing import LabelEncoder, StandardScaler
import pandas
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, precision_recall_curve, auc

x_train, x_test, y_train, y_test = train_test_split(X,y,random_state=42)



#param_dist = {'C': uniform(0.1, 100)}

#clf = SVC(kernel='rbf', verbose=True)
# Perform random search
#random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=10, cv=5)
#random_search.fit(x_train, y_train)
#print(x_train.shape)
#print(x_train)
#print(y_train.shape)
#print(y_train)
# Best C value
#best_C = random_search.best_params_['C']
#print("best C: ", best_C)

#94.44072358087371


svm_final_PCA = SVC(C=94.44, kernel='rbf', verbose=True)
svm_final_PCA.fit(x_train, y_train)
print("PCA MODEL FINISHED")
#y_pred = clf.predict(x_test)
#y_pred = to_categorical(y_pred)


In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn import metrics

# Assuming clf is your trained classifier and x_test is your test data
y_pred = svm_final_PCA.predict(x_test)
print(y_pred.shape)
# Convert to NumPy array if y_test is a Pandas Series and flatten if necessary
y_test_flat = y_test.to_numpy().flatten()

# Flatten the predicted values (if needed)
y_pred_flat = y_pred.flatten()

# Convert to binary classification (example)
# Adjust this step according to your specific problem
threshold = 0.5
y_pred_binary = (y_pred_flat > threshold).astype(int)

# Reshape y_test_flat to match the shape of y_pred_binary
y_test_binary = (y_test_flat[:len(y_pred_binary)] > threshold).astype(int)

# Calculate metrics
accuracy = accuracy_score(y_test_binary, y_pred_binary)
f1 = f1_score(y_test_binary, y_pred_binary, average='weighted')
precision = precision_score(y_test_binary, y_pred_binary, average='weighted')
recall = recall_score(y_test_binary, y_pred_binary, average='weighted')

# Confusion matrix
cm = confusion_matrix(y_test_binary, y_pred_binary)

# True Positive Rate (TPR) and False Positive Rate (FPR)
tpr_value = cm[1, 1] / (cm[1, 1] + cm[1, 0])
fpr_value = cm[0, 1] / (cm[0, 1] + cm[0, 0])

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision Score:", precision)
print("Recall Score:", recall)
print("TPR Score:", tpr_value)
print("FPR Score:", fpr_value)


In [ ]:
from sklearn.metrics import roc_curve, precision_recall_curve, auc

precision_a, recall_a, threshold = precision_recall_curve(y_test_binary, y_pred_binary)
prc_auc = auc(recall_a, precision_a)
print("Area Under the PR Curve score: ", prc_auc)


fpr, tpr, threshold = roc_curve(y_test_binary, y_pred_binary)
roc_auc = auc(fpr, tpr)
print("Area Under the ROC Curve score: ", roc_auc)

In [ ]:
import json

# Assuming tpr and fpr are NumPy arrays
print("Type of f1:", type(f1))
print("Type of precision:", type(precision))
print("Type of recall:", type(recall))
print("Type of tpr_value:", type(tpr_value))
print("Type of fpr_value:", type(fpr_value))
print("Type of roc_auc:", type(roc_auc))
print("Type of prc_auc:", type(prc_auc))
print("Type of tpr:", type(tpr))
print("Type of fpr:", type(fpr))

precision_list = precision_a.tolist()
recall_list = recall_a.tolist()
tpr_list = tpr.tolist()
fpr_list = fpr.tolist()

#print(f1.type())

data = {
    "Average F1 Score": f1, 
    "Average Precision": precision, 
    "Average Recall": recall, 
    "Average True Positive Rate": tpr_value, 
    "Average False Positive Rate": fpr_value, 
    "Average ROC AUC": roc_auc, 
    "Average PRC AUC": prc_auc,
    "TPR Array" : tpr_list,
    "FPR Array" : fpr_list,
    "Precision Array": precision_list, 
    "Recall Array": recall_list
}

# Specify the file path
file_path = "/kaggle/working/SVM - Synthetic Dataset with PCA.json"

# Write the data to a JSON file
with open(file_path, "w") as json_file:
    json.dump(data, json_file)